### Homework
### 李嘉琛 2301212345

#### Extract the first page of all pdfs in the "mypdfs " folder and merge them into one pdf file named "new.pdf".

In [5]:
from pathlib import Path

path = Path(r"mypdfs")
pdf_files = [p for p in path.glob("*.pdf")]
print(pdf_files)

[WindowsPath('mypdfs/1905.05406.pdf'), WindowsPath('mypdfs/1908.02426.pdf'), WindowsPath('mypdfs/2103.07579.pdf'), WindowsPath('mypdfs/2106.13112.pdf'), WindowsPath('mypdfs/2109.05673.pdf')]


In [6]:
# merge them into one pdf
from PyPDF4 import PdfFileReader, PdfFileMerger, PdfFileWriter
newfilelist = []
for file in pdf_files:
    with open(file, "rb") as pdf_file:
        pdf = PdfFileReader(pdf_file)
        page = pdf.getPage(0)
    
        writer = PdfFileWriter()
        writer.addPage(page)

        newfile = file.name[:-4]+'_new.pdf'
        newfilelist.append(newfile)
        with open(newfile, "wb") as output:
             writer.write(output)
print(newfilelist)

['1905.05406_new.pdf', '1908.02426_new.pdf', '2103.07579_new.pdf', '2106.13112_new.pdf', '2109.05673_new.pdf']


In [7]:
merger = PdfFileMerger()

for file_name in newfilelist:
    merger.append(file_name)
merger.write("new.pdf")

#### Take the abstracts of the first 20 papers in the given URL, translate them into Chinese, and write them into the file named 'my_abstract.txt’.  
#### URL = https://openaccess.thecvf.com/ICCV2021?day=2021-10-12

In [8]:
import requests
from bs4 import BeautifulSoup
import re
import random
from hashlib import md5
import time

In [26]:
def getUrl(url):
    response = requests.get(url)
    pdfUrl = []
    count = 0
    bs = BeautifulSoup(response.text,"html.parser")  # Parse the HTML content of the response using BeautifulSoup
    pdfs = bs.find_all(href=re.compile('/content/.*html')) # Find all anchor tags with href attribute matching the specified pattern
    names = []
    for tag in pdfs:
        if count < 20:  # Limit the count to 20
            pdfUrl.append("https://openaccess.thecvf.com"+tag.get("href"))
            count = count+1
            # Extract the name from the href attribute and modify it before appending to the names list
            names.append(re.split('[./]', tag.get("href"))[-2][:-16].replace("_"," ")) 
        else: break
    return names,pdfUrl


def getAbstract(paperUrls):
    abstract = []
    for url in paperUrls:
        response = requests.get(url)
        bs = BeautifulSoup(response.text,"html.parser")
        abstract.append(bs.find(id="abstract").text.strip()) #get the content of id "abstract"
    return abstract

def translate_api(input_text):
    # Set your own appid/appkey.
    appid = '20231228001923565'
    appkey = 'L0FUaa5T1WCTtXvCvW4R'
    def make_md5(s, encoding='utf-8'):
        return md5(s.encode(encoding)).hexdigest()
    # For list of language codes, please refer to `https://api.fanyi.baidu.com/doc/21`
    from_lang = 'en'
    to_lang = 'zh'
    endpoint = 'http://api.fanyi.baidu.com'
    path = '/api/trans/vip/translate'
    url = endpoint + path
    
    query = input_text
    # Generate salt and sign
    salt = random.randint(32768, 65536)
    sign = make_md5(appid + query + str(salt) + appkey)
    # Build request
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    payload = {'appid': appid, 'q': query, 'from': from_lang, 'to': to_lang, 'salt': salt, 'sign': sign}
    # Send request
    r = requests.post(url, params=payload, headers=headers)
    result = r.json()
    print(result)

    return result['trans_result'][0]['dst']

def write2txt(path,title,contents):
    with open(path,"w") as f:
        for ti,con in zip(title,contents):
            f.write(ti+"\n")
            f.write(con+"\n")
            f.write("\n")

In [27]:
web = "https://openaccess.thecvf.com/ICCV2021?day=2021-10-12"
name,pdfurls = getUrl(web)
abstracts = getAbstract(pdfurls)
ch_abstracts = []

In [28]:
for ab in abstracts:
    temp = translate_api(ab)
    ch_abstracts.append(temp)
    time.sleep(1)
write2txt("./my_abstract.txt",name,ch_abstracts)

{'from': 'en', 'to': 'zh', 'trans_result': [{'src': 'Learning-based image denoising methods have been bounded to situations where well-aligned noisy and clean images are given, or samples are synthesized from predetermined noise models, e.g., Gaussian. While recent generative noise modeling methods aim to simulate the unknown distribution of real-world noise, several limitations still exist. In a practical scenario, a noise generator should learn to simulate the general and complex noise distribution without using paired noisy and clean images. However, since existing methods are constructed on the unrealistic assumption of real-world noise, they tend to generate implausible patterns and cannot express complicated noise maps. Therefore, we introduce a Clean-to-Noisy image generation framework, namely C2N, to imitate complex real-world noise without using any paired examples. We construct the noise generator in C2N accordingly with each component of real-world noise characteristics to e